# Prophet

------------
## - 정의

In [ ]:
# - 페이스북에서 공개한 시계열 예측 라이브러리
# - 정확도가 높고 빠르며 직관적인 파라미터로 모델 수정이 용이
# - Prophet 모델의 주요 구성 요소 : (1) Trend / (2) Seasonality / (3) Holiday
# - 공식 : y(t) = g(t) + s(t) + h(t) + ϵi

# - g(t) : piecewise linear or logistic growth curve for modelling non-periodic changes in time series
# - s(t) : periodic changes (e.g. weekly/yearly seasonality)
# - h(t) : effects of holidays (user provided) with irregular schedules
# - ϵi: error term accounts for any unusual changes not accommodated by the model

# - Trend를 구성하는 g(t) 함수는 '주기적이지 않은 변화인 트렌드'를 나타내며, 부분적으로 선형 또는 logistic 곡선으로 이루어져 있음.
# - Seasonality인 s(t) 함수는 weekly, yearly 등 '주기적으로 나타나는 패턴'들을 포함
#   Prophet 알고리즘은 푸리에급수(Fourier series)를 이용하여 seasonality 패턴을 추정 (푸리에급수는 주기가 있는 함수를 삼각함수의 급수로 나타낸 것)
# - Holiday를 나타내는 h(t) 함수는 '휴일과 같이 불규칙한 이벤트'들을 나타내며, 만약 특정 기간에 값이 비정상적으로 증가 또는 감소했다면 holiday로 정의하여 모델에 반영할 수 있음.
# - 마지막으로 ϵi는 '정규분포라고 가정한 오차'임

# 사용 데이터셋(train_data)은 '날짜/수치' 컬럼으로 이루어진 데이터프레임 / 지수평활과 비슷한 구조

# 참고 : https://hyperconnect.github.io/2020/03/09/prophet-package.html

## - Parameter 설명

### (1) Trend

In [ ]:
# ## Trend
# # (1) changepoint_prior_scale 값을 높일수록 changepoint(Trend가 변화하는 지점)를 더 민감하게 감지하지만
# #     값을 너무 높이면 overfitting의 위험이 있음
# # (2) 트렌드가 바뀌는 시점, 즉 '서비스 확대 배포' 또는 '프로모션 등으로 인한 변화 시점'을 알고 있다면
# #     changepoints 파라미터를 추가할 수 있고
# #     n_changepoints로 changepoints 수를 지정할 수 있음
# #     (위의 두 파라미터를 설정해주지 않아도 모델이 자동으로 감지함)

# changepoints             : 트렌드 변화시점을 명시하는 리스트 값
# changepoint_prior_scale  : changepoint의 유연성 조절(default : 0.05)
# n_changepoints           : changepoint의 개수
# changepoint_range        : changepoint 설정 가능 범위(기본적으로 데이터 중 80% 범위 내에서 changepoint를 설정)

### (2) Seasonality

In [ ]:
# ## Seasonality
# # (1) seasonality :
# #   - 값을 높여주면 주기 패턴을 더 유연하게 잡아주지만 너무 유연하게 fitting 된 모델은 overfitting 위험이 높음
# #   - daily/weely/yearly seasonality에 대한 파라미터는 제공되지만, monthly는 제공되지 않지만 임의로 seasonality를 정의하여 모델에 반영할 수 있음
# #   - model.add_seasonality(name='monthly', period=30.5, fourier_order=5) / 주기가 30.5일, fourier order가 5인 ‘monthly’라는 명칭의 seasonality를 추가
# # (2) seasonality_mode :
# #   - Additive와 Multiplicative를 잘못 구분하면 오차항을 제대로 분리하지 못하게 됨
# #   - Additive : 데이터의 진폭이 일정함 (Additive Seasonality : Time series = Trend + Seasonality + Error)
# #   - Multiplicative : 데이터의 진폭이 점점 증가하거나 감소함 (Multiplicative Seasonality : Time series = Trend * Seasonality * Error)

# yearly_seasonality       : 연 계절성(default : 10 / 숫자값 또는 False로 조절)
# weekly_seasonality       : 주 계절성(default : 10 / 숫자값 또는 False로 조절)
# daily_seasonality        : 일 계절성(default : 10 / 숫자값 또는 False로 조절)
# seasonality_prior_scale  : 계절성 반영 강도
# seasonality_mode         : 'additive' or 'multiplicative' (default : additive)

### (3) Holiday

In [ ]:
# ## Holiday
# # (1) 데이터에 영향을 미치는 '휴일'이나 '프로모션 같은 이벤트'를 알고 있다면 모델에 반영하여 정확도를 높일 수 있음
# # (2) model.add_country_holidays(country_name='국가코드') : 국가 공휴일을 불러올 수 있음(하지만 모든 국가의 공휴일이 있는건 아님)
# # (3) holiday의 정보를 담은 데이터 프레임을 생성
# #     휴일이 휴일 (전/후)에도 영향을 미친다면, 해당 일 수 만큼 파라미터로 설정해 줄 수 있음
# #     예를 들어, 공휴일 영향이 그 다음날에도 영향을 미친다면 lower_window=0, upper_window=1 을 추가해줌
# # [holiday 정의 예시]
# # holiday = pd.DataFrame({'holiday': 'holiday',
# #                         'ds': pd.concat([pd.Series(pd.date_range('2017-05-05', '2017-06-03', freq='D')),
# #                                          pd.Series(pd.date_range('2018-05-05', '2018-06-03', freq='D')),
# #                                          pd.Series(pd.date_range('2019-05-05', '2019-06-03', freq='D')),
# #                                          pd.Series(pd.date_range('2020-05-05', '2020-06-03', freq='D'))])
# #                         lower_window = 0,
# #                         upper_window = 1})
# # [정의한 holiday 적용 예시]
# # m = Prophet(   
# #                 # trend
# #                 changepoint_prior_scale = 0.3,
# #                 # seasonality
# #                 weekly_seasonality = 10,
# #                 yearly_seasonality = 10,
# #                 daily_seasonality = False,
# #                 seasonality_mode = 'multiplicative',
# #                 # holiday
# #                 holidays = holiday,
# #                 holidays_prior_scale = 15
# #             )

# holidays             : 휴일 또는 이벤트 기간을 명시한 데이터프레임
# holiday_prior_scale  : holiday 반영 강도

---------

## 0. 환경설정

In [2]:
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools

import seaborn as sns
import matplotlib.pyplot as plt  # from matplotlib import pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

# !pip install pystan
# !pip install fbprophet

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
from fbprophet.plot import add_changepoints_to_plot
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

# # 그래프 폰트
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# rc('font', family=font_name)
# plt.rc('font', family='Malgun Gothic')
# plt.rcParams["figure.figsize"] = (8, 4)
# plt.rcParams['axes.unicode_minus'] = False

## 1. Modeling 예시

In [ ]:
# ## [예시 1]
# # 예측 건수 설정
# pred_periods = 100

# # 모델 생성
# model = Prophet()
# model.fit(train_data)

# # 예측 수행
# future = model.make_future_dataframe(periods = pred_periods)
# forecast = model.predict(future)

# # 예측 결과를 시각화
# fig1 = model.plot(forecast)

# # fitting된 모델의 컴포넌트들을 시각화
# fig2 = model.plot_components(forecast)

In [ ]:
# ## [예시 2]
# # add_changepoints_to_plot : 모델의 changepoint를 시각화 해보고, changepoint_prior_scale 값 변경에 따른 Trend 변화를 살펴봄
# model = Prophet(
#                     # trend
#                     changepoint_prior_scale=0.3,
#                     n_changepoints=7
#                     changepoint_range=1

#                     # seasonality
#                     yearly_seasonality=20,
#                     weekly_seasonality=10,
#                     daily_seasonality=False,
#                     seasonality_prior_scale = 15,
#                     seasonality_mode = 'multiplicative',
    
#                     # holiday
#                     holidays = holiday,
#                     holidays_prior_scale = 15
# )
# model.fit(train_data)
# future = model.make_future_dataframe(periods = pred_periods)
# forecast = model.predict(future)
# fig = model.plot(forecast)
# a = add_changepoints_to_plot(fig.gca(), m, forecast)  # gca()로 현재의 Axes를, gcf()로 현재의 Figure 객체를 구함 / 두 함수는 만약 현재의 Axes나 Figure가 없을 경우 새로 생성함

In [ ]:

            
##############################################################           

# 모델 적합
df_prophet = Prophet(changepoint_prior_scale = 0.15, daily_seasonality = True)
df_prophet.fit(df)

# 향후 1년간의 time stamp 생성
예측을 위해서는 예측하고자 하는 날짜가 ds 컬럼에 있어야함
Prophet.make_future_dataframe을 사용하여 지정된 날짜 수만큼 미래로 확장되는 적절한 데이터 프레임을 얻을 수 있음
기본적으로 모형의 훈련에 사용된 시계열의 날짜도 포함되므로 모델이 맞는지 확인할 수 있음

fcast_time = 90
df_forecast = df_prophet.make_future_dataframe(periods = fcast_time, freq = 'D')  # make_future_dataframe() 함수를 이용해 예측값을 넣을 데이터 프레임을 생성합니다. 이때, 인자로 들어가는 periods 값은 향후 몇일 (또는 주,월 등 단위 주기) 을 예측할 것인지를 의미
df_forecast.tail(10)
df_forecast = df_prophet.predict(df_forecast)

# 예측 결과 확인
df_forecast[['ds','yhat',..]].tail()

# 예측값 시각화
df_prophet.plot(df_forecast, xlabel = 'Data', ylabel = 'Price($)')

## 예측 구성요소 확인
# 예측에 사용된 구성 요소는 Prophet.plot_components 메소드를 사용하여 확인 가능
# 기본적으로 시계열의 추세, 연간 계절성, 주간 계절성이 표시됨
df_prophet.plot_components(df_forecast)
plt.show()

# 교차검증
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(df_prophet, initial = '1095 days', period = '180 days', horizon = '365 days')

# 모형 성능 확인
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

# 교차검증 결과 시각화
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric = 'mae')

##############################################################           
            
    # prophet
    model = Prophet(yearly_seasonality=10, weekly_seasonality=False, daily_seasonality=False, 
                    changepoint_range=1, changepoint_prior_scale=0.1, n_changepoints=7)
    
#     # 모델의 Trend를 조절할 수 있는 파라미터는 다음과 같습니다.
#     changepoints : 트렌드 변화시점을 명시한 리스트값
#     changepoint_prior_scale : changepoint(trend) 의 유연성 조절
#     n_changepoints : changepoint 의 개수
#     changepoint_range : changepoint 설정 가능 범위. (기본적으로 데이터 중 80% 범위 내에서 changepoint를 설정합니다.)
    
    
# 모델의 changepoint 를 시각화해보고, changepoint_prior_scale 값 변경에 따른 Trend 변화를 살펴보겠습니다.
# changepoint_prior_scale = 0.05 (default)
# 빨간 실선은 트렌드를 의미하며, 빨간 점선은 트렌드가 변화하는 changepoint 를 의미
from fbprophet.plot import add_changepoints_to_plot

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

# changepoint_prior_scale = 0.3 인 경우
m = Prophet(changepoint_prior_scale=0.3)
m.fit(df)

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

# changepoint_prior_scale 값을 0.3으로 높여준 후 트렌드를 더 유연하게 감지하는 것을 확인할 수 있습니다.
# 이 값을 너무 높여버리면 overfitting의 위험이 있으니 주의해야 합니다.
# 만약 트렌드가 바뀌는 시점(서비스 확대 배포 또는 프로모션 등으로 인한 변화 시점)을 알고 있다면, changepoints 파라미터를 추가할 수 있음 (= 트렌드 바뀌는 시점을 리스트로 직접 지정)
# changepoints 수 또한 n_changepoints 로 지정할 수 있습니다. 물론, 이 두 파라미터를 설정해주지 않아도 모델이 자동으로 감지합니다.




    
    
#     prophet = Prophet()
    model.fit(df_hap[['ds', 'y']])
                      
    # 5년 예측
    future = model.make_future_dataframe(periods=5, freq='y')  #  periods 값은 향후 몇일 (또는 주,월 등 단위 주기) 을 예측할 것인지를 의미
    forecast = model.predict(future)

    # 예측값 시각화
    fig1 = model.plot(forecast)
    
    # 피팅된 모델의 컴포넌트들을 시각화
    fig2 = model.plot_components(forecast)
    
    # 컴포넌트 시각화 해석
    # => 만약 모델이 데이터의 Trend를 잘 잡아내지 못하는 것 같다면, changepoint_prior_scale 파라미터값을 높여주어 changepoint를 더 민감하게 감지하도록 할 수 있습니다. 여기서 changepoint란, Trend가 변화하는 지점을 의미
    # => 아래 차트는 각각 ‘주 계절성’과 ‘연 계절성’을 의미
    # => Trend와 마찬가지로 Seasonality또한 seasonality_prior_scale 파라미터로 모델 반영 강도를 조절할 수 있습니다.
    
    
    
    
    
    
    
    
    
    
    
    
    # 학생 수는 음의 값이 나올 수 없기 때문에 -인 경우 0으로 변경
    forecast.yhat = forecast.yhat.apply(lambda x: 0 if x <= 0 else x)
    
    fore = forecast.loc[:, ['ds', 'yhat']]
    fore.columns = ['ds', 'PRED_VAL']
#     fore_df = fore[fore.ds <= '2020-12-31']
    result2 = fore.copy(deep=True)
    
    # 실제값이랑 예측값 합치기
    df_real = df_hap21[df_hap21['SIDO_EDU_NM'] == s]
    df_real = df_real[['BASE_YY', 'y']]
    df_real.columns = ['BASE_YY', 'REAL_VAL']
    
    result2['BASE_YY'] = result2['ds'].dt.year.astype('str')
    result2 = result2.drop('ds', axis=1)
    
    df_pred_real = result2.merge(df_real, on='BASE_YY', how='outer')
    df_pred_real['시도교육청'] = s
    df_pred_real['교육지원청'] = sg
    df_pred_real['학교구분'] = d
    
    df_pred_real = df_pred_real[['BASE_YY','시도교육청','교육지원청','학교구분','REAL_VAL','PRED_VAL']]
    df_result = df_result.append(df_pred_real)
    df_result['PRED_VAL'] = np.where(df_result['PRED_VAL'] < 0, 0, df_result['PRED_VAL'])
#     result2['BASE_YY'] = pd.DatetimeIndex(result2['ds']).year.astype('str')
#     result2 = result2.drop(['ds'], axis=1)
    
    # 정확도
    df_result['acc'] = round(
    1 - abs((df_result['REAL_VAL'] - df_result['PRED_VAL']) / df_result['REAL_VAL'])) * 100

    
    # 그래프
    title = '%s %s %s' % (s,sg,d)
     # plt = get_prophet_plot(prophet, forecast, title)
    plt = get_line_plot(df_pred_real, 'BASE_YY', ['REAL_VAL', 'PRED_VAL'])
    plt.title(title, size=20)
    plt.savefig(res_path + 'prophet_%s.png' % (title))
    plt.close()
    
    df_sub_acc = df_pred_real[~df_pred_real['BASE_YY'].isin(['2021', '2022', '2023', '2024', '2025'])]
    r2 = r2_score(df_sub_acc['REAL_VAL'], df_sub_acc['PRED_VAL'])
            
    se = np.square(df_result['REAL_VAL'] - df_result['PRED_VAL'])
    mse = np.mean(se)
    rmse = np.sqrt(mse)
    df_result['mse'] = se
        # print('Mean Squared Error(MSE)--------', MSE)
        
    # 오류는 안나는데 데이터가 비정상적으로 많이 출력
#     model_parameters = model_parameters.append(pd.DataFrame({'시도': s, '교육지원청명':sg , '학교구분': d ,'결정계수':r2,'MSE' : mse}, index=[0]))

            

In [ ]:
result2['BASE_YY'] = result2['ds'].dt.year.astype('str')
result2 = result2.drop('ds', axis=1)
result2